In [ ]:
%load_ext autoreload
%autoreload 2
from openmsimodel.db.open_db import OpenDB
from openmsimodel.graph.open_graph import OpenGraph
from openmsimodel.workflow.old_birdshot_workflow import BIRDSHOTWorfklow
from openmsimodel.utilities.graph_tools import BODisplay
import networkx as nx
import json
from pathlib import Path
curr_pwd = Path().parent.absolute()

# Workflow

In [ ]:
root= curr_pwd / "input/data/AAB" # put data and Sample Data folders inside input
output = curr_pwd / "output/AAB"
iteration = "AAB"
aab_synthesis_path = curr_pwd / "input/Sample Data/Iteration2_AAB/HTMDEC AAB Summary Synthesis Results.xlsx"
aab_srjt_path = curr_pwd / "input/Sample Data/Iteration2_AAB/Copy of HTMDEC AAB-VAM NI SRJT Set Summary.xlsx"
aab_birdshot_workflow = BIRDSHOTWorfklow(root, output=output,iteration=iteration,synthesis_path=aab_synthesis_path, srjt_path=aab_srjt_path)
aab_birdshot_workflow.build()

### Dive into

Terminal material of a workflow, which allows us to trace the entire workflow history

In [ ]:
aab_birdshot_workflow.terminal_material

List of logically grouped subworkflows in the entire workflow

In [ ]:
print(list(aab_birdshot_workflow.subs.keys()))
print("nb of blocks: {}".format(len(aab_birdshot_workflow.subs.keys())))

Manual workflows, which are not sciency but 

In [ ]:
manual_blocks = [obj.name for obj in aab_birdshot_workflow.subs.values() if obj.type=="manual"]
print(manual_blocks)
print(len(manual_blocks))

In [ ]:
arc_melting_aab04_vam_a_all = aab_birdshot_workflow.subs['Arc Melting of Alloy Alloy (AAB04-VAM-A)'].return_all()
print(arc_melting_aab04_vam_a_all)
arc_melting_aab04_vam_a_all_gemd = aab_birdshot_workflow.subs['Arc Melting of Alloy Alloy (AAB04-VAM-A)'].return_all_gemd()
print(len(arc_melting_aab04_vam_a_all_gemd))
aab16_vam_a_tensile_T09_all_gemd = aab_birdshot_workflow.subs['Set Alloy (AAB16-VAM-A) T. Sample for Tensile (T09) charact.'].return_all_gemd()

### multiple dumping approaches

In [ ]:
aab_birdshot_workflow.thin_dumps(aab_birdshot_workflow.terminal_material, overwrite=True) # dumps into a single folder using a terminal GEMD object and GEMD dump methods 

In [ ]:
# aab_birdshot_workflow.thin_structured_dumps(overwrite=True) # dumps into a nice COMPOSITION_ID/FABRICATION_METHOD/BATCH structure, with in-house methods

# Viz

In [ ]:
assets_to_add = {
            "add_attributes": 1,
            "add_file_links": 1,
            "add_tags": 1,
        }
output = curr_pwd / "output/AAB"

### visualize the whole workflow...

In [ ]:
dirpath = curr_pwd / "output/AAB/terminal_history/thin"
open_graph = OpenGraph("unstructured",dirpath=dirpath,output=output)
all_G, all_relabeled_G, all_name_mapping = open_graph.build_graph(
    assets_to_add=assets_to_add,
    add_separate_node=0,
    which='run',
)
open_graph.launch(all_relabeled_G)

### ... or an important workflow branch...

In [ ]:
dirpath = curr_pwd / "output/AAB/structured/AAB01/VAM/A/thin_jsons"
open_graph = OpenGraph("aab01_vam_a_thin",dirpath=dirpath,output=output)
G, relabeled_G, name_mapping = open_graph.build_graph(
    assets_to_add=assets_to_add,
    add_separate_node=0,
    which='run',
)
open_graph.launch(relabeled_G)

### ... or a subworkflow...

In [ ]:
# open_graph = OpenGraph("arcmelting_block_aab04_vam_a", dirpath=arc_melting_aab04_vam_a_all_gemd+aab16_vam_a_tensile_T09_all_gemd,output=output, 
#     restrictive=True)
open_graph = OpenGraph("arcmelting_block_aab04_vam_a", dirpath=arc_melting_aab04_vam_a_all_gemd,output=output, 
    restrictive=True)
G, relabeled_G, name_mapping = open_graph.build_graph(
    assets_to_add=assets_to_add,
    add_separate_node=0,
    which='run',
)
# print(name_mapping.keys())
# relabeled_G.remove_node(name_mapping["7b24a6ce-3de3-452b-b48a-f8e3e026b829"])
open_graph.launch(relabeled_G)

### ... or a a BaseElement...

In [ ]:
# open_graph = OpenGraph("arcmelting_block_aab04_vam_a", dirpath=arc_melting_aab04_vam_a_all[1].return_all_gemd()
#                         ,output=output, restrictive=True)
open_graph = OpenGraph("aab16_vam_a_tensile_T09", dirpath=[aab16_vam_a_tensile_T09_all_gemd[-1]],output=output, restrictive=True)
G, relabeled_G, name_mapping = open_graph.build_graph(
    assets_to_add=assets_to_add,
    add_separate_node=0,
    which='run',
)
open_graph.launch(relabeled_G)

### ... or a single elements with its descendents and predecessors

In [ ]:
import random
identifier = random.choice(list(all_name_mapping.keys()))
print(f"Name: {all_name_mapping[identifier]}")
print(f"Identifier: {identifier}")
functions = [nx.descendants, nx.ancestors]
try: # depending on the randomly selected element
    identifier_G = OpenGraph.extract_subgraph(all_G, identifier, func=functions)
    identifier_G = OpenGraph.map_to_graphviz(identifier_G, all_name_mapping)
    open_graph.launch(identifier_G)
except Exception as e:
    print(e)

TODO: SVG & Dot Viewer

In [ ]:
import graphviz
# dot_path = curr_pwd / "output/input_legend.dot"
# /srv/hemi01-j01/openmsimodel/examples/birdshot/output/AAB/unstructured_run_graph.dot
dot_path = curr_pwd / "output/AAB/unstructured_run_graph.dot"
graph = graphviz.Source.from_file(dot_path)
from IPython.display import Image
path = graph.render(dot_path, format='jpg')
Image(path)

# DB

In [ ]:
database_name='GEMD'
private_path='/home/arachid1/.private/gemd_db.json'
db = OpenDB(database_name=database_name, private_path=private_path,output=curr_pwd / "output/queries_log")
model_id =11

In [ ]:
query = f""" select distinct * from GEMDModel """
models = db.gemd_db.execute_query(query)
db.print_and_dump(models, query, "models", dump=False)


In [ ]:
import re

# query = f""" select distinct * from GEMDModel """
# Returns a dictionary of values from the GEMDObject JSON context
def fn(row):
    data = json.loads(row)
    
    # Parse the name into the batch, iteration, fabrication method, location
    (identifier, location) = re.findall('\((.*?)\)', data['name'])
    (code, method, batch) = identifier.split('-') # AAB14-VAM-A
    iteration = code[0:3]

    output = {
        'Batch': batch,
        'Iteration': iteration,
        'Method': method,
        'Code': code,
        'Location': location
    }
    output.update({_["name"]:_["value"]["nominal"] for _ in data["properties"]})
    return output
query = """
select *
from gemdobject 
where ISJSON(context) > 0
and JSON_VALUE(context, '$.tags[0]') LIKE 'composition_id::AAB%'
and JSON_VALUE(context, '$.name') LIKE 'Tensile charact%'
and gemd_type = 'measurement_run'
"""
models = db.gemd_db.execute_query(query)
# db.print_and_dump(models, query, "models", dump=False)
models.apply(lambda row: fn(row.context), axis='columns', result_type='expand')


Test query retrieving all measurement runs

In [ ]:
query = f""" SELECT * FROM GEMDObject WHERE model_id={model_id} AND gemd_type='measurement_run' """
_all_measurements = db.gemd_db.execute_query(query)
db.print_and_dump(_all_measurements, query, "_all_measurements", dump=False)

In [ ]:
from gemd.json import GEMDJson
import json
# print(type(_all_measurements.loc[0]['context']))
# _all_measurements['context'] = _all_measurements['context'].apply(lambda x: x[0])
# GEMDJson().thin_dumps(_all_measurements['context'])
BODisplay(df=_all_measurements)

In [ ]:
c = _all_measurements.iloc[1]['context']
print(json.dumps(json.loads(c), indent=3))

a couple of invalid jsons that need fixing

In [ ]:
query = f""" SELECT *
FROM GEMDObject 
WHERE model_id={model_id} and isjson(context)=0
"""
test = db.gemd_db.execute_query(query)
db.print_and_dump(test, query, "invalid_jsons")

first advanced Birdshot query

In [ ]:
# query = f""" SELECT 
#         i.uid, properties.name, properties.type, properties.nominal_value, properties.categorical_value, properties.units, properties.value, SplitJSON.*
#         FROM gemdobject i 
#         CROSS APPLY OPENJSON(JSON_QUERY(i.context, '$.properties')) 
#         WITH 
#         (
#                 name VARCHAR(50) '$.name', 
#                 value [nvarchar](max) '$.value' as json,
#                 nominal_value [nvarchar](4000) '$.value.nominal',
#                 categorical_value [nvarchar](4000) '$.value.category',
#                 units [nvarchar](4000) '$.value.units',
#                 type [nvarchar](32) '$.value.type'
#         )
#         AS properties 
#         CROSS APPLY (
#         SELECT
#                 SUBSTRING(value, 1, CHARINDEX('::', value) - 1) AS column_name,
#                 SUBSTRING(value, CHARINDEX('::', value) + 2, LEN(value)) AS column_value
#         FROM OPENJSON(JSON_QUERY(i.context, '$.tags'))
#         ) AS SplitJSON
#         WHERE i.model_id={model_id} AND i.gemd_type='measurement_run' AND isjson(i.context)!=0"""

In [ ]:
birdshot = db.gemd_db.execute_query(query)
db.print_and_dump(birdshot, query, "birdshot")

query to curate measurements and measurement data, for optimal use by to birdshot

In [ ]:
# query="""DECLARE @model_id INT; -- Define the model_id parameter
# SET @model_id = [YourModelIDValue]; -- Set the value of model_id

# SELECT
#     i.uid,
#     properties.name, 
#     properties.type,
#     properties.nominal_value, 
#     properties.categorical_value,
#     properties.units,
#     properties.value,
#     -- Add columns from the JSON tags array
#     MAX(CASE WHEN column_name = 'composition_id' THEN column_value END) AS composition_id,
#     MAX(CASE WHEN column_name = 'batch' THEN column_value END) AS batch,
#     MAX(CASE WHEN column_name = 'fabrication_method' THEN column_value END) AS fabrication_method,
#     -- Add columns from the JSON properties object
#     MAX(CASE WHEN property_name = 'name' THEN property_value END) AS property_name,
#     MAX(CASE WHEN property_name = 'json' THEN property_value END) AS property_json,
#     MAX(CASE WHEN property_name = 'nominal' THEN property_value END) AS property_nominal,
#     MAX(CASE WHEN property_name = 'category' THEN property_value END) AS property_category,
#     MAX(CASE WHEN property_name = 'units' THEN property_value END) AS property_units,
#     MAX(CASE WHEN property_name = 'type' THEN property_value END) AS property_type
# FROM GEMDOBject i
# -- Split the JSON tags array and add columns
# CROSS APPLY OPENJSON(JSON_QUERY(i.context, '$.tags'))
# WITH (
#     column_name NVARCHAR(50) '$' -- The tag contains both name and value
# ) AS tags_data
# CROSS APPLY (
#     SELECT 
#         SUBSTRING(tags_data.column_name, 1, CHARINDEX('::', tags_data.column_name) - 1) AS column_name,
#         SUBSTRING(tags_data.column_name, CHARINDEX('::', tags_data.column_name) + 2, LEN(tags_data.column_name)) AS column_value
# ) AS parsed_tags
# -- Extract properties from the JSON properties object
# CROSS APPLY OPENJSON(JSON_QUERY(i.context, '$.properties'))
# WITH (
#     property_name VARCHAR(50) '$.name', 
#     property_value [nvarchar](max) '$.value' as json,
#     property_nominal [nvarchar](4000) '$.value.nominal',
#     property_category [nvarchar](4000) '$.value.category',
#     property_units [nvarchar](4000) '$.value.units',
#     property_type [nvarchar](32) '$.value.type'
# ) AS properties
# WHERE i.model_id = @model_id
#     AND i.gemd_type = 'measurement_run'
#     AND ISJSON(i.context) = 1 -- Use '= 1' for ISJSON to check for valid JSON
# GROUP BY
#     i.uid,
#     properties.name, 
#     properties.type,
#     properties.nominal_value, 
#     properties.categorical_value,
#     properties.units,
#     properties.value; -- Group by the JSON property"""


In [ ]:
# advanced_birdshot = db.gemd_db.execute_query(query)
# db.print_and_dump(advanced_birdshot, query, "advanced_birdshot")

TODO: further tailoring of query to specific measurements of interest for Bayesian Optimization